In [1]:
# Import all required libraries
import os
import gdown
import zipfile
import cv2
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import tensorflow as tf
import numpy as np
from tensorflow.keras.preprocessing import image
from tensorflow.keras import models, layers
from tensorflow.keras.preprocessing.image import ImageDataGenerator

: 

In [ ]:
url = "https://drive.google.com/file/d/1X-RXCt4kvl7v9vtwPqNilc6Zxi5E3THL/view?usp=sharing"
filename = 'SITUATION.zip'
gdown.download(url, output=filename, quiet=False, use_cookies=True, fuzzy=True)

In [3]:
with zipfile.ZipFile(filename, 'r') as zip_ref:
    zip_ref.extractall("/content")

In [ ]:
image_dir = "/content/SITUATION/"
train_dir = os.path.join(image_dir,'TRAIN')
test_dir = os.path.join(image_dir,'TEST')

print("Train directory -> ", train_dir)
print("Test directory -> ", test_dir)

In [ ]:
train_Ok_dir = os.path.join(train_dir,'OK')
train_NotOk_dir =os.path.join(train_dir,'NOT OK')
train_NotInRange_dir =os.path.join(train_dir,'NOT IN RANGE')

print("OK directory -> ", train_Ok_dir)
print("NOT OK directory -> ", train_NotOk_dir)
print("NOT IN RANGE directory -> ", train_NotInRange_dir)

In [ ]:
image_files_Ok = os.listdir(train_Ok_dir)
image_files_NotOk = os.listdir(train_NotOk_dir)
image_files_NotInRange = os.listdir(train_NotInRange_dir)

print("Images found OK -> ", image_files_Ok[:5]) # slicing the data upto 5 images
print("Images found NOT OK -> ", image_files_NotOk[:5]) # slicing the data upto 5 images
print("Images found NOT IN RANGE -> ", image_files_NotInRange[:5]) # slicing the data upto 5 images

In [ ]:
plt.figure(figsize=(10,10))
for i in range(3):
    ax = plt.subplot(3, 3, i + 1)
    mg_path = os.path.join(train_Ok_dir, image_files_Ok[i])
    img = mpimg.imread(mg_path)  # Load the image from file path
    plt.imshow(img)
    plt.title(image_files_Ok[i])
    plt.axis("off")

In [8]:
# Normalize each image values between 0 and 1.
trainDataGen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=30,
    zoom_range=0.2,
    horizontal_flip = True,
    fill_mode='nearest',
    shear_range = 0.5
)

testDataGen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=30,
    zoom_range=0.2,
    horizontal_flip = True,
    fill_mode='nearest',
    shear_range = 0.5
)

In [ ]:
training = trainDataGen.flow_from_directory(train_dir, target_size=(150, 150)) # Provie the train directory to model.
training.class_indices                                                         # Labeling to the data set by creating JSON.

In [ ]:
testing = testDataGen.flow_from_directory(test_dir, target_size=(150, 150))
testing.class_indices

In [ ]:
# Get one batch of images and labels
images, labels = next(training)

# Plot the images
plt.figure(figsize=(10,10))
for i in range(3):
    ax = plt.subplot(3, 3, i + 1)
    plt.imshow(images[i])         # Already normalized [0,1]
    plt.title(np.argmax(labels[i]))  # If categorical, get class index
    plt.axis("off")
plt.tight_layout()
plt.show()

In [12]:
model=models.Sequential()                             # Sequential layering.
model.add(layers.Conv2D(32,(3,3),activation='relu'))  # Create window to standardize picxels.
model.add(layers.MaxPooling2D((2,2)))                 # Used for controlling over fitting.
model.add(layers.Conv2D(64,(3,3),activation='relu'))
model.add(layers.MaxPooling2D((2,2)))
model.add(layers.Conv2D(128,(3,3),activation='relu'))
model.add(layers.MaxPooling2D((2,2)))
model.add(layers.Flatten())
model.add(layers.Dense(512,activation='relu'))
model.add(layers.Dense(3,activation='softmax'))       # For creating 3 outputs

In [13]:
model.compile(optimizer=tf.keras.optimizers.RMSprop(learning_rate=0.001), loss='categorical_crossentropy',metrics=['accuracy'])

In [ ]:
history=model.fit(training, validation_data=testing, steps_per_epoch=8, epochs=15, verbose=1) # Repeat 15 times to refine accuracy

In [ ]:
model.save('models/scene-detect.h5')

In [ ]:
model.summary() # Summarize the layering details

In [ ]:
from google.colab import files
up = files.upload()

In [ ]:
# Load the image
img_path = "Chilling.jpeg"  # <- your test image path
img = image.load_img(img_path, target_size=(150, 150, 3))  # Match input size of model

# Convert and preprocess image
img_array = image.img_to_array(img)
img_array = img_array / 255.0  # Normalize
img_array = np.expand_dims(img_array, axis=0)  # Add batch dimension

# Predict
predictions = model.predict(img_array)
confidence = np.max(predictions[0])  # Highest probability
predicted_class = np.argmax(predictions[0])  # Index of prediction

# Map class index to label
class_indices = training.class_indices  # Dictionary from training (e.g., {'OK': 0, 'NOT OK': 1, ...})
labels = dict((v, k) for k, v in class_indices.items())  # Invert it
predicted_label = labels[predicted_class]

# Load original image using OpenCV
cv_img = cv2.imread(img_path)
cv_img = cv2.resize(cv_img, (800, 600))  # Resize for display

# Determine color based on predicted label
if predicted_label == "OK":
    color = (0, 255, 0)  # Green
elif predicted_label == "NOT OK":
    color = (0, 0, 255)  # Red
else:
    color = (0, 0, 0)  # Black

# Prepare text
text = f"{predicted_label} ({confidence * 100:.2f}%)"
font = cv2.FONT_HERSHEY_SIMPLEX
font_scale = 1.2
thickness = 3
text_position = (30, 60)

# Draw background rectangle for better visibility
(text_width, text_height), _ = cv2.getTextSize(text, font, font_scale, thickness)
cv2.rectangle(cv_img,
              (text_position[0] - 10, text_position[1] - 40),
              (text_position[0] + text_width + 10, text_position[1] + 10),
              (255, 255, 255),
              -1)  # White background

# Now add the text on top of the rectangle
cv2.putText(cv_img, text, text_position, font, font_scale, color, thickness)

# Convert BGR (OpenCV) to RGB (matplotlib)
cv_img_rgb = cv2.cvtColor(cv_img, cv2.COLOR_BGR2RGB)

# Show with matplotlib
plt.figure(figsize=(10, 6))
plt.imshow(cv_img_rgb)
plt.axis('off')
plt.title("Prediction Result")
plt.show()